<a href="https://colab.research.google.com/github/golesuman/66daysofdata/blob/main/Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is the colaborative filtering


In [1]:
import pandas as pd
import numpy as np
from io import StringIO
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
!unzip Collaborative\ Filtering\ Dataset.zip

Archive:  Collaborative Filtering Dataset.zip
replace dataset/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dataset/links.csv       
  inflating: __MACOSX/dataset/._links.csv  
  inflating: dataset/tags.csv        
  inflating: __MACOSX/dataset/._tags.csv  
  inflating: dataset/ratings.csv     
  inflating: __MACOSX/dataset/._ratings.csv  
  inflating: dataset/README.txt      
  inflating: __MACOSX/dataset/._README.txt  
  inflating: dataset/movies.csv      
  inflating: __MACOSX/dataset/._movies.csv  


In [3]:
text="""action1,action2,action3,romantic1,romantic2,romantic3
user 1,4,5,3,,2,1
user 2,5,3,3,2,2,
user 3,1,,,4,5,4
user 4,,2,1,4,,3
user 5,1,,2,3,3,4"""

In [4]:
df=pd.read_csv(StringIO(text),delimiter=',',index_col=0)
df.head()

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,4.0,5.0,3.0,NaN,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,NaN
user 3,1.0,NaN,NaN,4.0,5.0,4.0
user 4,NaN,2.0,1.0,4.0,NaN,3.0
user 5,1.0,NaN,2.0,3.0,3.0,4.0


In [5]:
df=df.fillna(0)
df

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,4.0,5.0,3.0,0.0,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,0.0
user 3,1.0,0.0,0.0,4.0,5.0,4.0
user 4,0.0,2.0,1.0,4.0,0.0,3.0
user 5,1.0,0.0,2.0,3.0,3.0,4.0


In [6]:
def standardize(row):
  new_row = (row-row.mean()) / (row.max() - row.min())
  return new_row

In [7]:
ratings_std=df.apply(standardize)
ratings_std

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,0.36,0.6,0.400000,-0.65,-0.08,-0.35
user 2,0.56,0.2,0.400000,-0.15,-0.08,-0.60
user 3,-0.24,-0.4,-0.600000,0.35,0.52,0.40
user 4,-0.44,0.0,-0.266667,0.35,-0.48,0.15
user 5,-0.24,-0.4,0.066667,0.10,0.12,0.40


In [10]:
item_similarity = cosine_similarity(ratings_std.T)
print(item_similarity)


[[ 1.          0.70668875  0.81368151 -0.79941088 -0.02539184 -0.91410609]
 [ 0.70668875  1.          0.72310153 -0.84515425 -0.5189993  -0.84337386]
 [ 0.81368151  0.72310153  1.         -0.84794611 -0.3799803  -0.80218063]
 [-0.79941088 -0.84515425 -0.84794611  1.          0.14803913  0.72374686]
 [-0.02539184 -0.5189993  -0.3799803   0.14803913  1.          0.39393939]
 [-0.91410609 -0.84337386 -0.80218063  0.72374686  0.39393939  1.        ]]


In [12]:
item_similarity_df=pd.DataFrame(item_similarity,index=df.columns,columns=df.columns)
item_similarity_df.head()

,action1,action2,action3,romantic1,romantic2,romantic3
action1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
action2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
action3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
romantic1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
romantic2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939


In [26]:
## Lets make recommendations
def get_similar_movies(movie_name,user_rating):
  similar_score = item_similarity_df[movie_name]*(user_rating-2.5)
  similar_score = similar_score.sort_values(ascending=False)
  return similar_score


print(get_similar_movies('romantic3',1))

action1      1.371159
action2      1.265061
action3      1.203271
romantic2   -0.590909
romantic1   -1.085620
romantic3   -1.500000
Name: romantic3, dtype: float64


In [23]:
action_lover = [('action1',5), ('romantic2',1), ('romantic3',1)]
similar_movies=pd.DataFrame()

for movie,rating in action_lover:
  similar_movies = similar_movies.append(get_similar_movies(movie,rating),ignore_index=True)

similar_movies.head()

,action1,action3,action2,romantic2,romantic1,romantic3
0,2.500000,2.034204,1.766722,-0.063480,-1.998527,-2.285265
1,0.038088,0.569970,0.778499,-1.500000,-0.222059,-0.590909
2,1.371159,1.203271,1.265061,-0.590909,-1.085620,-1.500000


In [25]:
similar_movies.sum().sort_values(ascending=False)

action1      3.909247
action2      3.810282
action3      3.807445
romantic2   -2.154389
romantic1   -3.306206
romantic3   -4.376174
dtype: float64